<a href="https://colab.research.google.com/github/tuomaseerola/music_and_science_seminar/blob/master/seminar2A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music and Science – Tutorial on Audio Analysis

[Tuomas Eerola](https://www.durham.ac.uk/staff/tuomas-eerola/), Durham University, Music Department, 2024.

For **Music and Science** Module, Seminar 4

_Last update 26/02/2024 by T. Eerola_

# Section 1: Preliminaries

Welcome to Colab. This is a notebook that can run python code within a cloud service and offer the results within the same web page. You can edit any code blocks and try out what happens. Note that your changes will not have permanent impact on the notebook, as this notebook will be reset when you leave the page.

We are going to go through the basics of analysing audio examples. There will be 5 _Learning Tasks_ to prompt you to try out the analysis your self and the tasks will become more difficult towards the end.

To start the analysis, click **play button** in the top left corner of the code below. This will set up the system for this notebook. [Librosa](https://librosa.org/doc/0.8.1/) is the audio analysis tool (created by Brian McFee and others, see their [paper](https://conference.scipy.org/proceedings/scipy2015/pdfs/brian_mcfee.pdf)) that we will be using and the rest of the libraries that are activated here are for convenience and creating graphs.

In [ ]:
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
print(librosa.__version__)

# Section 2: Read an audio file

## Section 2.1: Read Librosa example excerpts

Here are some example excerpts that come with Librosa. We can use any of them in the subsequent sections. Just remove the hashtag (#) in front of the line to get that audio file.

The following code also shows how to select only a _segment_ of an audio file. This is done by keywords `offset` and `duration`. Offset specifies where you want to start the segment (in seconds) and duration (also in seconds) is self-explanatory.

In [ ]:
#filename = librosa.ex('trumpet')   # Mihai Sorohan - Monophonic trumpet recording
#filename = librosa.ex('brahms')    # Hungarian Dance no. 5
#filename = librosa.ex('choice')    # A short drum and bass loop
filename = librosa.ex('fishin')     # Karissa Hobbs - Let’s Go Fishin’ A folk/pop song with verse/chorus/verse structure and vocals
#filename = librosa.ex('nutcracker')# Tchaikovsky - Dance of the Sugar Plum Fairy
#filename = librosa.ex('vibeace')   # 60-second clip
x, sr = librosa.load(filename, duration=20) # if you want to start the segment from a specific point, add offset = [2 for example] in the command

plt.figure(figsize=(12, 4))                  # create figure
librosa.display.waveshow(y = x,sr = sr)      # plot waveform
ipd.display(ipd.Audio(data = x, rate = sr))  # create playback object

---
### _Learning Task 1_
---
_Try out some audio examples yourself. To select one, remove the hashtag (`#`) from the beginning of the line. NB. Only one audio example should be run at a time, so make sure that the rest of the 'filename' commands have a # in front of them._

## Section 2.2: Load audio files from online sources
This is how to load example sound files from GitHub pages (or anywhere from online, if you know the URL).

In [ ]:
import soundfile as sf
import io
from six.moves.urllib.request import urlopen

url = "https://raw.githubusercontent.com/tuomaseerola/audio/master/clar_one_note.wav"
#url = "https://raw.githubusercontent.com/tuomaseerola/audio/master/trumpet_one_note.wav"
#url = "https://raw.githubusercontent.com/tuomaseerola/audio/master/harp_one_note.wav"
#url = "https://raw.githubusercontent.com/tuomaseerola/audio/master/harpsichord_one_note.wav"

x, sr = sf.read(io.BytesIO(urlopen(url).read()))

plt.figure(figsize=(12, 4))
librosa.display.waveshow(y = x,sr = sr, color = 'purple')
ipd.display(ipd.Audio(data = x, rate = sr))

## Section 2.3: Load audio files from your computer / Google Drive (optional)
It is also possible to load any audio you have on your hard drive to the workspace of the notebook by clicking the folder icon (on the left) and choosing the file, waiting for the file to upload, and then referring to the file. These audio files will **NOT be saved** across the sessions unless you connect your Google Drive with Colab, which is easy (see [External data: Local Files, Drive, Sheets and Cloud Storage](https://colab.research.google.com/notebooks/io.ipynb)).

In [ ]:
# NOTE! I have now commented these three lines out. If you want to use this idea, uncomment (remove the # signs from lines 2-4)
#x, sr = librosa.load('my_own_recording.wav') # if you want to start the segment from a specific point, add offset = [2 for example] in the command
#plt.figure(figsize=(12, 4))
#ipd.display(ipd.Audio(data = x, rate = sr))

# Section 3: Explore acoustic features across time
First we extract RMS (Root Mean Square) energy of an audio file and convert this into decibels.

In [ ]:
filename = librosa.ex('nutcracker')
x, sr = librosa.load(filename, duration = 20, offset = 2)
ipd.display(ipd.Audio(data = x, rate = sr))

rms = librosa.feature.rms(y = x)                   # Extract dynamics (RMS)
feat = librosa.amplitude_to_db(rms, ref = np.max)  # Convert into dB. Note that this is a relative measure (loudest is now 0)
#feat = librosa.feature.spectral_centroid(x)       # Calculate spectral centroid
times = librosa.times_like(feat)                   # Create X-axis for time in seconds

plt.figure(figsize=(12, 4))                        # Create the plot placeholder
plt.plot(times, feat[0], color =  'black')         # Plot feature across time
plt.show()

---
### _Learning Task 2_
---
_There is another feature called _spectral centroid_ that is often used as a proxy for perceptual brightness that you could visualise across time. Calculation of this is very simple and shown above (commented out). You could explore it or some other features from [spectral features](https://librosa.org/doc/latest/feature.html). Note that the labelling of the Y axis needs to be fixed._

# Section 4: Analyse pitch
This is best done with a monophonic example, although `librosa` has algorithm to track all pitches as well. Here we extract the fundamental frequency of the trumpet solo using a probabilistic variant of the so-called YIN method. The variant is by Mauch and Dixon (2014) and the original technique was proposed by De Cheveigne and Kawahara (2002).

In [ ]:
filename = librosa.ex('trumpet')              # Mihai Sorohan - Monophonic trumpet recording
x, sr = librosa.load(filename)
ipd.display(ipd.Audio(data = x, rate = sr))   # code added to create playback object

# This is where the YIN algorithm is applied. fmin refers to the lower frequency threshold and fmax to higher frequency threshold.
f0, voiced_flag, voiced_probs = librosa.pyin(x, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C6'))
times = librosa.times_like(f0)

# Plot the fundamental frequency, make the y-axis logarithmic for easier interpretation
plt.figure(figsize=(12, 4))
plt.plot(times, f0, 'ro:', linewidth = 2)
plt.xlabel('Time')
plt.ylabel('Hz')
ax=plt.gca()
ax.yaxis.set_major_formatter(librosa.display.LogHzFormatter())
ax.set(yticks=[320,320+80,320+80*2,320+80*3,320+80*4,320+80*5])
ax.set(ylim=[300, 800])
ax.yaxis.set_major_formatter(librosa.display.LogHzFormatter())

# Section 5: Analyse spectrum

Spectrum is a useful representation of audio that shows the energy across frequencies. Try out plotting the spectrogram with the trumpet sound.

## Section 5.1: Spectrogram

In [ ]:
filename = librosa.ex('trumpet') #PROMPT: Try this out with the trumpet audio example. What is the spectrogram telling you?
x, sr = librosa.load(filename)  #for the "example1d" file, put a # in front of this line of code

#url = "https://raw.githubusercontent.com/annaliesemg/audio/master/example1d.wav" #PROMPT: Try out this other example. Which frequencies are the most prominent (lower ones or higher ones)?
#x, sr = sf.read(io.BytesIO(urlopen(url).read()))

stft = np.abs(librosa.stft(x))
freqs = librosa.fft_frequencies(sr = sr)

ipd.display(ipd.Audio(data = x, rate = sr))
plt.figure(figsize=(12, 4))
librosa.display.specshow(librosa.amplitude_to_db(stft, ref = np.max), x_axis = 'time', y_axis = 'log')
plt.show()

## Section 5.2 Spectrum
Here we take only the last note of the trumpet example. Which frequency has the highest amplitude? First we extract the fundamental frequency and print the results out.

In [ ]:
filename = librosa.ex('trumpet')                               # Mihai Sorohan - Monophonic trumpet recording
x, sr = librosa.load(filename, offset = 2.6, duration = 0.75)  # Take the last note (from 2.6 to 3.35 s)
### OR USE ANOTHER AUDIO FILE BY UNCOMMENTING THE NEXT 2 LINES  ####
#url = "https://raw.githubusercontent.com/tuomaseerola/audio/master/clar_one_note.wav"
#x, sr = sf.read(io.BytesIO(urlopen(url).read()))

ipd.display(ipd.Audio(data=x, rate=sr))                        # Create the playback option
stft = np.abs(librosa.stft(x))                                 # Short-term Fourier transform to get the frequencies
# Estimate fundamental frequency using an algorithm (pyin)
f0,voiced_flag, voiced_probs = librosa.pyin(x, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
f = np.nanmedian(f0)                                           # Get the Hz of the fundamental
n = librosa.hz_to_note(f)                                      # Convert Hz to note name
print('Fundamental is', round(f,1),'Hz which is', n )          # print

Now display the spectrum of this note.

In [ ]:
plt.figure(figsize=(12, 4))
# collapse across time and plot a spectrum representation (energy across frequencies)
Dmean = stft.mean(axis=1)/max(stft.mean(axis=1))
plt.plot(freqs, Dmean, color = 'm')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.xlim([300, 2000])             # Constrain X-axis (Hz)
x=np.arange(f,f*10,f)             # Put labels to the multiples of the fundamental
plt.xticks(x)
plt.show()

---
### _Learning Task 3_
---
_What does the spectrum of **clarinet** or **harpsichord** or **harp** look like? What tones are they playing? Tip: you can just alter the beginning of the 4.2 and uncomment the section that loads files from internet._



# Section 6: Analyse onsets and tempo
This section estimates the strength of **possible onsets** and then detects the onsets that are stronger than a threshold defined in the algorithm. There is also a beat tracking that uses the onsets to estimate the plausible steady beats in the audio. This information is then used to infer tempo (in the next section).

## Section 6.1 Onsets and steady beat tracking
Look at the "Choice" example at first. Take the first 20 seconds (as duration) of this file. Afterwards, have a look at the 'brahms' example as well.

In [ ]:
filename = librosa.ex('choice')                                             # Try this example first
#filename = librosa.ex('brahms')                                            # Hungarian Dance no. 5
#filename = librosa.ex('nutcracker')                                        # Nutcracker
x, sr = librosa.load(filename, duration = 10)

o_env = librosa.onset.onset_strength(y = x, sr = sr, aggregate = np.median,fmax=8000)      # Strength of onsets
times = librosa.times_like(o_env, sr = sr)                                  # Create timeline
onset_frames = librosa.onset.onset_detect(onset_envelope = o_env, sr = sr)  # Detect onsets

plt.figure(figsize=(12, 4))
librosa.display.waveshow(y = x,sr = sr, color = 'green')
plt.xlim([0,8])

plt.figure(figsize=(12, 4))
plt.plot(times, o_env, label = 'Onset strength')
plt.vlines(times[onset_frames], 0, o_env.max(), color = 'r', alpha = 0.9,
           linestyle='--', label='Onsets')
plt.legend()

# Sonify the detected beat events
tempo, beats = librosa.beat.beat_track(y=x, sr=sr,trim=True,onset_envelope=o_env, tightness=150.0)
y_beats = librosa.clicks(frames=beats, sr=sr)

combined = (x[0:len(y_beats)]+y_beats)/2
librosa.display.waveshow(y=abs(y_beats)*max(o_env),sr=sr,color='y')
plt.ylim([0,20])
plt.xlim([0,8])
plt.legend()
plt.show()
ipd.display(ipd.Audio(data=combined, rate=sr))

---
### _Learning Task 4_
---
_What are we seeing here? Can you explain the difference between onset strength (blue), detected onsets (red dotted lines) and beats (yellow)? Explore how these features behave in other examples (TIP: beginning of the code in Section 5.1 has the other examples commented out). How well are the onset detected and is the beat structure always what you think it should be?_


## Section 6.2 Analyse tempo

---
### _Learning Task 5_
---

_Find **tempo** of the file that you analysed above. The tempo is indicated in Beats Per Minute (BPM).
*Tip. There is a command called* `beat.tempo` *that can be used to calculate the tempo.* The following script will not work until you have completed the function in the first line._

In [ ]:
tempo = librosa.b...(x, aggregate = np.median)
print(np.round(tempo,1),'BPM')